# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 
from collections import Counter
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [6]:
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные
dvach = open('2ch_corpus.txt', encoding = 'utf-8').read()
news = open('lenta.txt', encoding = 'utf-8').read()

In [3]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


In [4]:
from nltk.tokenize import sent_tokenize
def ngrammer3(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

def ngrammer2(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [50]:
#sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach[:5000000])]
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [55]:
sentences_news = sentences_news[24:]

In [66]:
# Немного странный способ, но так я точно вижу, какие предложения я использую для расчета перплексии
perp = """Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. 
«Русский инвалид», 16 сентября 1914 года.Министерство народного просвещения, в виду происходящих чрезвычайных событий, признало соответственным в день годовщины со дня рождения М.Ю. Лермонтова (2-го октября 1914 года) ограничиться совершением в учебных заведениях панихиды по поэту, отложив празднование юбилея до более благоприятного времени.  
«Русский инвалид», 16 сентября 1914 года.Штабс-капитан П. Н. Нестеров на днях, увидев в районе Желтиева, в Галиции, летящий над нашим расположением австрийский аэроплан, собиравшийся бросить бомбы, взлетел на воздух, атаковал неприятеля и протаранил неприятельский аппарат, предотвратив жертвы в наших войсках. Сам Нестеров при этом погиб смертью героя. По словам доставленных в Киев пленных австрийских офицеров, всей неприятельской армии хорошо известно имя Нестерова. Во время воздушных разведок русских авиаторов австрийцы всегда безошибочно определяли, каким аппаратом управлял Нестеров. Когда показывался аэроплан-птица, красиво и вольно паривший в воздухе, австрийцы указывали — Das ist Nesteroff! Австрийцы боялись покойного, и все их усилия были направлены к прекращению его деятельности. За задержание отважного летчика была объявлена большая премия. Нестеров погиб в 27 лет. После Нестерова остались жена и двое детей — девочка, 5-ти лет, и мальчик, 3-х лет. Иллюстрированный журнал «Искры» № 35, сентябрь 1914 года  Песнь о Нестерове В безбрежности неба, в бескрайности ясной пустыниСражались лишь птицы и гибли лишь птицы доныне.Так птице подобный, полет свой направив могучий,Унесся пилот легкокрылый за темные тучи.А там, на земле, колыхаясь, знамена шумели, И лязгали сабли, и пушки зловеще гремели.И пламенно богу войны посылал он молитвы,И очи горели и звали и жаждали битвы…Величие духа в равнине обманчиво-зыбкой.Презрение смерти под светлою солнца улыбкой…Мольбы не напрасны. Велению рока послушный,Противник уж реет кругами в стихии воздушнойИ дрогнуло сердце. Рука, как пружина стальная,На руль налегла."""
len(perp)

2832

In [8]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer2(sentence))
    trigrams_news.update(ngrammer3(sentence))
  


In [9]:
# матрица 
matrix_news = lil_matrix((len(bigrams_news), 
                         len(unigrams_news)))

# к матрице нужно обращаться по индексам
# поэтому зафиксируем порядок слов в словаре и сделаем маппинг id-слово и слово-id
id2word = list(unigrams_news)
word2id = {word:i for i, word in enumerate(id2word)}

id2bigram = list(bigrams_news)
bigram2id = {bigram:i for i, bigram in enumerate(id2bigram)}


In [10]:
bigram2id

{'<start> <start>': 0,
 '<start> решенье': 1,
 'решенье созреломгновенье': 2,
 'созреломгновенье –': 3,
 '– и': 4,
 'и врезался': 5,
 'врезался дерзко': 6,
 'дерзко он': 7,
 'он в': 8,
 'в хрупкое': 9,
 'хрупкое тело': 10,
 'тело пощады': 11,
 'пощады не': 12,
 'не знает': 13,
 'знает стихия': 14,
 'стихия <end>': 15,
 '<start> победа': 16,
 'победа наградой': 17,
 'наградой …': 18,
 '… и': 19,
 'и пали': 20,
 'пали противники': 21,
 'противники грузной': 22,
 'грузной и': 23,
 'и страшной': 24,
 'страшной громадой': 25,
 'громадой …': 26,
 '… в': 27,
 'в безбрежности': 28,
 'безбрежности неба': 29,
 'неба в': 30,
 'в бескрайности': 31,
 'бескрайности ясной': 32,
 'ясной пустынисражались': 33,
 'пустынисражались лишь': 34,
 'лишь птицы': 35,
 'птицы и': 36,
 'и гибли': 37,
 'гибли лишь': 38,
 'птицы доныне': 39,
 'доныне <end>': 40,
 '<start> журнал': 41,
 'журнал «': 42,
 '« нива': 43,
 'нива »': 44,
 '» №': 45,
 '№ 37': 46,
 '37 сентябрь': 47,
 'сентябрь 1914': 48,
 '1914 daily': 49,

In [12]:

bigrams_news['<start> <start>']

32225

In [11]:
# заполняем матрицу
for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    # на пересечение биграмма и униграмма ставим вероятность встретить слово после биграммы
    matrix_news[bigram2id[bigram], word2id[word3]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram])
    

In [13]:
print(matrix_news[:5, :5].toarray())

[[0.00000000e+00 3.10318076e-05 0.00000000e+00 0.00000000e+00
  3.66175330e-03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]]


In [14]:
def generate(matrix, id2word, bigram2id, n=50, start='<start> <start>'):
    text = []  
    current_bigram = start
    
    for i in range(n):  
        current_bigram_idx = bigram2id[current_bigram]  
        
        chosen_word_idx = np.random.choice(matrix.shape[1], p=matrix[current_bigram_idx].toarray()[0]) 
        chosen_word = id2word[chosen_word_idx] 
        text.append(chosen_word) 
        
        current_bigram = current_bigram.split()[1] + ' ' + chosen_word
        
        if chosen_word == '<end>':  
            current_bigram = '<start> <start>' 
    return ' '.join(text)

In [15]:
print(generate(matrix_news, id2word, bigram2id).replace('<end>', '\n'))

как стало известно риа новости в прокуратуре также сообщил что по его словам в результате проведенной операции боевики были выбиты боевики 
 террористическая угроза в россии 
 в качестве примера пагубности имперской политики 
 на втором штурмовике су-25 следовал за этим тайваньские военные специалисты в области интернета и выяснили что


In [18]:
print(generate(matrix_news, id2word, bigram2id).replace('<end>', '\n'))

обсуждались в частности говорится с 00 00 1 января 2000 года регистрация доменного имени business com наиболее точно отражает суть ее деятельности в адвокатском бюро и московском направлениях перешли на усиленный режим несения службы 
 однако они специализируются главным образом живыми деньгами за поставки теплоэлектроэнергии является приоритетным направлением работы компании


In [19]:
print(generate(matrix_news, id2word, bigram2id).replace('<end>', '\n'))

нато да русские нет 
 представители ответчика заявили что в первом туре будет считаться претендент на пост президента отправил на имя некоего андрея батурина 
 сайт vote com информирует пользователя о том как идет доработка бюджета-2000 
 позвонившая утверждает что два израильских гражданина находящиеся в длительных служебных командировках а также


In [20]:
print(generate(matrix_news, id2word, bigram2id).replace('<end>', '\n'))

напомним что санкции оон против ирака введенных после войны в корее и 1 тысячу кубометров газа который добывается в самой компании миком 
 кроме того был передан ссср в счет погашения кредита вэб что означает производства в два раза дешевле а год назад и сразу переводились дальше на другие проблемы


In [16]:
def perplexity(logp, N): 
    return np.exp((-1/N) * logp)


def compute_join_trigram(text, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer3(['<start>', '<start>'] + tokens + ['<end>']): 
        word1, word2, word3 = ngram.split()
        bigram = word1 + ' ' + word2
        if bigram in bigram_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigram])
        else:
            prob += np.log(2e-5) 
    
    return prob, len(tokens)

In [63]:
news[:200]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые кали'

In [67]:
ps_trigram = []
for sent in sent_tokenize(news[:2832]): 
    prob, N = compute_join_trigram(sent, bigrams_news, trigrams_news)
    if not N:
        continue
    ps_trigram.append(perplexity(prob, N))

np.mean(ps_trigram)

96998.52897833694

In [68]:
#Проверила для себя, что это именно те предложения 
news[:2832]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. \n«Русский инвалид», 16 сентября 1914 года.Министерство народного просвещения, в виду происходящих чрезвычайных событий, признало соответств

In [23]:
phrase = '7 декабря прошла церемония вручения «ТЭФИ» — российской национальной премии за высшие достижения в области телевизионных искусств.'
log_prob, N = compute_join_trigram(phrase, bigrams_news, trigrams_news)
perplexity(log_prob, N)

29290.483435187627

In [24]:
phrase = """Один парень был проклят ведьмой, и из-за этого мог говорить лишь одно слово в год. И вот однажды он влюбился в девушку. Он не говорил 9 лет, а потом подошел к той самой девушке, и сказал:
- Я тебя люблю, выходи за меня.
- А зачем 9 лет ждал, тут же всего 6 слов было?
- Бля, чет тупанул."""
log_prob, N = compute_join_trigram(phrase, bigrams_news, trigrams_news)
perplexity(log_prob, N)

48677.33148748901

In [69]:
phrase = """Я прочитал уже 18 томов лайт новеллы, тоже метался от лолли к лолли, все таки принцесса лучше всех."""
log_prob, N = compute_join_trigram(phrase, bigrams_news, trigrams_news)
perplexity(log_prob, N)

68284.26057116111

## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.